# Natural Language Processing Homework 1


<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
    <title>توضیح متن و تسک انجام شده</title>
</head>
<body>
    <p dir="rtl">
        در این تسک، ابتدا داده‌های مربوط به اخبار را در چهار دسته‌بندی ورزشی، اقتصادی، سیاسی و اجتماعی جمع‌آوری میکنیم. منبع داده‌های من از سایت رویداد ۲۴ میباشد و صفحات مختلف آن را توسط کتابخانه scrapy در پایتون، کرال کردم.
        <br>
        در ادامه از هر خبر عنوان و خلاصه آن خبر را استخراج کردم. حدود ۳.۷ مگابایت داده‌اولیه در اختیار داریم که باید آن را پردازش کنیم.
        <br>
        تسکی که به صورت کلی میخواهم روی داده‌ها انجام دهم بدین شکل هست که ابتدا عبارات کلیدی متن را استخراج نمایم. سپس بر اساس ۴ دسته بندی مختلف این کار را انجام دهم و این ترکیب‌ها را با یکدیگر مقایسه نمایم. </p>
</body>
</html>


## install and import requirements

In [2]:
!pip install hazm 
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import json
from hazm import *
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import os

In [40]:
with open('news.json', 'r') as file:
  import json
  base_data = json.load(file)
  data = [(item['title'] + "\n" +item['content']) for item in base_data]

## Pre-Processing

In [35]:
data = [item.replace('\xa0', ' ') for item in data]
data

['قیمت جدید گوشت قرمز اعلام شد\nقائم مقام وزیر جهاد کشاورزی با اینکه مشکلی در تامین گوشت نداریم، گفت: بنابر برنامه ریزی صورت گرفته واردات گوشت قرمز به طور منظم در حال انجام است.',
 'کنفرانس خبری با کله و پاچه!\nرئیس اتحادیه اغذیه و پروتئین گلپایگان برای اثبات حرف های خود تخلف برخی از طباخان این شهرستان با کله و پاچه به نشست خبری برگزار کرد.',
 'قیمت روز گوشی موبایل ۱۲ آبان ۱۴۰۲\nقیمت روز گوشی موبایل در بازار ۱۲ آبان ۱۴۰۲ را در جداول زیر ببینید.',
 'قیمت روز خودرو ۱۲ آبان ۱۴۰۲\nقیمت روز خودرو در بازار ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.',
 'قیمت روز ارز های دیجیتال ۱۲ آبان ۱۴۰۲\nقیمت روز ارز های دیجیتال در بازار ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.',
 'قیمت دلار، طلا و سکه امروز ۱۲ آبان ۱۴۰۲\nقیمت دلار، طلا و سکه در بازار امروز ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.',
 'قیمت روز خودرو های خارجی؛ ماشینی که هر روز ۱۵۰ میلیون گران می شود!\nقیمت روز و جدید انواع خودرو های خارجی در تاریخ ۱۲ آبان را می توانید در ادامه ببینید.',
 'تمام نگاه  دولت به شرق است/ هیچ پاسخی برای وضعیت اقتصادی و معیشتی م

<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
</head>
<body>
    <p dir="rtl">
    در این بخش نرمالایز و توکنایز را انجام میدهیم.
    </p>
</body>
</html>


### Normalize

In [7]:
normalizer = Normalizer()
news_normalized = [normalizer.normalize(item)for item in data]


for x in random.sample(news_normalized, 10):
    print(''.join(x))


قیمت دلار، طلا و سکه امروز ۱۱ آبان ۱۴۰۲
قیمت دلار، طلا و سکه در بازار امروز ۱۱ آبان ۱۴۰۲ را در جدول زیر ببینید.
چهره باورنکردنی بوکسور ایرانی پس از ناکامی از حریف هندی
ایمان رمضان‌پور بوکسور سنگین کشورمان برابر حریف هندی خود شکست خورد وی این در این رابطه گفت: پیش از بازی‎های آسیایی فقط یک اردوی مشترک داشتیم اما شکست مقابل حریف هندی را توجیه نمی‌کنم.
هشدار روزنامه جمهوری اسلامی درباره تهدیدات حضور مهاجران افغانستانی در ایران
پاکستان علیرغم حمایت‌های اولیه‌اش از طالبان اکنون آن را تهدیدی بزرگ می‌داند، ولی شما با اینکه هر روز حادثه‌ای تلخ در نقاط مختلف کشورمان که به دست اعزامی‌های طالبان رخ می‌دهد می‌بینید هنوز هم حاضر نیستید دست از تطهیرشان بردارید!
پشت پرده تحت لیسانس‌های داخلی؛ کاورهای خالی از دارو! 
اخیرا ویدیویی در فضای مجازی منتشر شده که کاورهای خالی از دارو در ورق‌های کپسول به مردم داده می‌شود.
رفتار تحسین‌برانگیز صاحب ماشین برای خواسته دو نوجوان
اخیرا ویدیویی در فضای مجازی منتشر شد که رفتار درست و انسانی صاحب ماشین را برای درخواست دو نوجوان نشان داد که موجب تحسین کاربران شد.
دریاچ

### Tokenize

word tokenize and then sentence tokenize.

In [8]:
news_sentences = [sent_tokenize(item) for item in news_normalized]
news_tokens = [[word_tokenize(sent) for sent in sents] for sents in news_sentences]


for y in [item[0] for item in random.sample(news_tokens, 20) if item]:
    print(' '.join(y))

حزب‌الله لبنان : کشور‌های غربی و آمریکا ، ملت فلسطین و عرب‌ها را آدم نمی‌دانند رئیس شورای اجرایی حزب‌الله لبنان در واکنش به حمله رژیم صهیونیستی به بیمارستان در نوار غزه و تهدیدات دشمن اعلام کرد : « پاسخ به اشتباهی که می‌توانید مرتکب شوید ، قاطع خواهد_بود » و طرح اخراج مردم غزه هرگز اجرایی نخواهد_شد .
سقوط وحشتناک جرارد پیکه از روی سکو‌ها بازیکن سابق بارسلونا که در حال حاضر مشغول برگزاری مسابقاتی به نام کینگز لیگ که خودش آن را راه‌اندازی کرده_است ، به خاطر زمین خوردن عجیبش سوژه شده_است .
پیش‌بینی بازار مسکن در نیمه دوم سال بازار مسکن در رکود کامل به سر می‌برد و ماه‌هاست معاملات در این بازار به کمترین میزان خود رسیده_است .
پیش‌بینی بورس سه‌شنبه ۹ آبان ۱۴۰۲ بورس تهران ، پس از لمس کف هشت ماهه ارزش دلاری بازار ، با اهرم ارزندگی قیمت‌ها و تلاش‌های سازمان بورس برای حمایت از بازار ، بازگشت .
تهران طوفانی / هوای این مناطق سرد می‌شود سازمان هواشناسی با صدور ۲ هشدار زرد از رگبار باران ، رعد و برق وزش باد شدید موقت و کاهش نسبی دما و در بخشی از کشور و همچنین وزش باد شدید و خیزش گرد و خاک در بخش دیگ

### Frequency Analysis
<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
</head>
<body>
    <p dir="rtl">
        در این بخش فرکانس کلمات مختلف را بررسی مینماییم.
    </p>
</body>
</html>


In [9]:
from nltk import FreqDist
import itertools
import pandas as pd


stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stop_words.txt','r','utf-8').readlines()]
all_tokens = list(itertools.chain(*itertools.chain(*news_tokens)))
all_tokens_non_stop = [t for t in all_tokens if t not in stopwords and len(t) > 1]

dataframe_nonstop = {}

dataframe_nonstop['rouydad24'] = FreqDist(all_tokens_non_stop).most_common(50)

freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop)    
freq_analysis_nonstop

,rouydad24
0,"(ایران, 628)"
1,"(قیمت, 615)"
2,"(روز, 491)"
3,"(بازار, 462)"
4,"(۱۴۰۲, 417)"
5,"(اسرائیل, 411)"
6,"(پرسپولیس, 355)"
7,"(غزه, 354)"
8,"(تهران, 329)"
9,"(کشور, 316)"


### Lemmatizer

In [36]:

lemmatizer = Lemmatizer()
all_tokens_nonstop_lemstem =   [lemmatizer.lemmatize(t) for t in all_tokens_non_stop]

all_tokens_lemstem =   [lemmatizer.lemmatize(t) for t in all_tokens]
dataframe_nonstop_lemstem = {}

dataframe_nonstop_lemstem['rouydad24.ir'] = FreqDist(all_tokens_nonstop_lemstem).most_common(50)

freq_analysis_nonstop_lemstem = pd.DataFrame(dataframe_nonstop_lemstem)   
freq_analysis_nonstop_lemstem

,rouydad24.ir
0,"(ایران, 736)"
1,"(قیمت, 670)"
2,"(روز, 563)"
3,"(بازار, 491)"
4,"(اسرائیل, 444)"
5,"(کشور, 436)"
6,"(۱۴۰۲, 417)"
7,"(پرسپولیس, 368)"
8,"(غزه, 358)"
9,"(تهران, 329)"


### Extract Most Common Nouns, Verbs, Adverbs and Adjectives
we do this by tagging every token and then analyzing the frequency of each word.

In [13]:
tagger = POSTagger(model='./resources/pos_tagger.model')

In [15]:
import copy
dataframe_nonstop_lemstem_advjj = {}
pss=set()
selected = {"صفت ها": [], "قید ها" : [], "فعل ها" : [], "اسم ها" : []}
for x,y in FreqDist(all_tokens_nonstop_lemstem).most_common(1000):
    pos = tagger.tag([x])[0][1]
    pss.add(pos)
    if pos in ['ADJ']:
        selected['صفت ها'].append((x,y)) 
    elif pos in ['NOUN']:
        selected['اسم ها'].append((x,y)) 
    elif pos in ['ADV']:
        selected['قید ها'].append((x,y)) 
    elif pos in ['VERB']:
        selected['فعل ها'].append((x,y)) 
for key, value in selected.items() :
  dataframe_nonstop_lemstem_advjj[key]= copy.deepcopy(value[0:8])
dataframe_nonstop_lemstem_advjj = pd.DataFrame(dataframe_nonstop_lemstem_advjj) 
dataframe_nonstop_lemstem_advjj   

{'ADJ', 'NOUN,EZ', 'ADP', 'CCONJ', 'ADP,EZ', 'NOUN', 'ADJ,EZ', 'PRON', 'ADV', 'NUM', 'VERB'}


,صفت ها,قید ها,فعل ها,اسم ها
0,"(ملی, 198)","(امروز, 225)","(گذاشت#گذار, 72)","(ایران, 736)"
1,"(آزاد, 113)","(گذشته, 126)","(چیست, 37)","(قیمت, 670)"
2,"(عجیب, 111)","(تازه, 32)","(کیست, 24)","(روز, 563)"
3,"(منتشر, 94)","(آخر, 25)","(برد, 22)","(بازار, 491)"
4,"(دیجیتال, 93)","(امسال, 24)","(کشته, 20)","(اسرائیل, 444)"
5,"(جهانی, 80)","(چقدر, 23)","(لرزاند, 20)","(کشور, 436)"
6,"(سیاسی, 70)","(کنون, 20)","(رسیده, 14)","(پرسپولیس, 368)"
7,"(شدید, 69)","(فردا, 18)","(معتقدند, 14)","(غزه, 358)"


## Most Common Phrases In Rouydad24.

In [29]:
def vasf_ext(sentence):
    grammar = r"""
      NVASF: {<NOUN|ADV><ADJ>}
      GHEIDV: {<ADV><VERB>}
      EZAFI: {<NOUN><NOUN|ADV>+}
    """
    cp = nltk.RegexpParser(grammar)
    return (cp.parse(sentence))


mosufs = []
gheids = []
ezafi = []
for sentences in news_tokens:
  for sentence in sentences:
    tree=vasf_ext(tagger.tag([word for word in sentence]))
    for subtree in tree.subtrees():
        if subtree.label() == 'NVASF':
            mosufs.append(subtree.leaves())
        if subtree.label() == 'EZAFI':
            ezafi.append(subtree.leaves())  

for sentences in news_tokens:
  for sentence in sentences:
    tree=vasf_ext(tagger.tag([lemmatizer.lemmatize(word) for word in sentence]))
    for subtree in tree.subtrees():
        if subtree.label() == 'GHEIDV':
            gheids.append(subtree.leaves())


In [30]:
results = dict()
results['ترکیب های وصفی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in mosufs]).most_common(50)
results['قید ها و فعل های مرکب'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in gheids]).most_common(50)
results['ترکیب های اضافی'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in ezafi]).most_common(50)
results['کل عبارات'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in ezafi + gheids + mosufs]).most_common(50)
pd.DataFrame(results)

,ترکیب های وصفی,قید ها و فعل های مرکب,ترکیب های اضافی,کل عبارات
0,"(لیونل مسی, 6)","(چقدر #است, 12)","(هدف قرار, 17)","(هدف قرار, 17)"
1,"(بسیار زیادی, 4)","(چه بود#باش, 8)","(کریستیانو رونالدو, 17)","(کریستیانو رونالدو, 17)"
2,"(ادامه پیدا, 4)","(چه شد#شو, 5)","(واکنش نشان, 16)","(واکنش نشان, 16)"
3,"(اعدام محکوم, 4)","(قبل بود#باش, 4)","(یحیی گل‌محمدی, 15)","(یحیی گل‌محمدی, 15)"
4,"(ویدئو زیر, 4)","(پیش رفت#رو, 4)","(آتیلا پسیانی, 13)","(آتیلا پسیانی, 13)"
5,"(تهران بیهوش, 4)","(چرا باید, 4)","(سردار آزمون, 10)","(چقدر #است, 12)"
6,"(غزه آواره, 4)","(باز کرد#کن, 3)","(علیرضا بیرانوند, 9)","(سردار آزمون, 10)"
7,"(دست پیدا, 4)","(چه کرد#کن, 3)","(امام خمینی, 8)","(علیرضا بیرانوند, 9)"
8,"(تصاویر منتشر, 4)","(بیشتر شد#شو, 3)","(رشیدی کوچی, 7)","(امام خمینی, 8)"
9,"(اتفاقی عجیب, 4)","(بس کرد#کن, 3)","(آیت‌الله خامنه‌ای, 7)","(چه بود#باش, 8)"


## مقایسه عبارات پر تکرار بین کتگوری‌های مختلف

<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
</head>
<body>
    <p dir="rtl">
در این بخش فانکشنالیتی های مختلف را ترکیب میکنیم تا برای هر دسته‌از متن به صورت جدا مقایسه را انجام دهیم    </p>
</body>
</html>

In [64]:
def extract_phrases(data):
    news_normalized = [normalizer.normalize(item)for item in data]
    news_sentences = [sent_tokenize(item) for item in news_normalized]
    news_tokens = [[word_tokenize(sent) for sent in sents] for sents in news_sentences]
    mosufs = []
    gheids = []
    ezafi = []
    for sentences in news_tokens:
      for sentence in sentences:
        tree=vasf_ext(tagger.tag([word for word in sentence]))
        for subtree in tree.subtrees():
            if subtree.label() == 'NVASF':
                mosufs.append(subtree.leaves())
            if subtree.label() == 'EZAFI':
                ezafi.append(subtree.leaves())  
    
    for sentences in news_tokens:
      for sentence in sentences:
        tree=vasf_ext(tagger.tag([lemmatizer.lemmatize(word) for word in sentence]))
        for subtree in tree.subtrees():
            if subtree.label() == 'GHEIDV':
                gheids.append(subtree.leaves())
    return mosufs, gheids, ezafi

## استخراج عبارات به ازای هر دسته‌بندی

In [65]:
sport_data = [(item['title'] + "\n" +item['content']) for item in base_data if item['category'] == 'sport']
economic_data = [(item['title'] + "\n" +item['content']) for item in base_data if item['category'] == 'economic']
politic_data = [(item['title'] + "\n" +item['content']) for item in base_data if item['category'] == 'politic']
society_data = [(item['title'] + "\n" +item['content']) for item in base_data if item['category'] == 'society']

In [66]:
sport_results = extract_phrases(sport_data)
economic_results = extract_phrases(economic_data)
politic_results = extract_phrases(politic_data)
society_results = extract_phrases(society_data)

## مقایسه ترکیب‌های وصفی

In [79]:
results = dict()
results['ورزشی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in sport_results[0]]).most_common(10)
results['اقتصادی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in economic_results[0]]).most_common(10)
results['سیاسسی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in politic_results[0]]).most_common(10)
results['اجتماعی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in society_results[0]]).most_common(10)
results['کل دسته‌بندی‌ها'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in sport_results[0] + economic_results[1] + politic_results[0] + society_results[0]]).most_common(10)
pd.DataFrame(results)

,ورزشی,اقتصادی,سیاسسی,اجتماعی,کل دسته‌بندی‌ها
0,"(لیونل مسی, 6)","(سوال مطرح, 3)","(اعدام محکوم, 3)","(اتفاقی عجیب, 4)","(چقدر #است, 10)"
1,"(حضور پیدا, 4)","(کاهش پیدا, 3)","(ایران بیش, 3)","(تهران بیهوش, 3)","(لیونل مسی, 6)"
2,"(ایران مشخص, 3)","(تقریبا بی‌اثر, 2)","(اسرائیل مدعی, 3)","(تازگی منتشر, 3)","(ویدئو زیر, 4)"
3,"(ویدئو زیر, 3)","(مبالغی برخوردار, 2)","(ادامه پیدا, 3)","(تصاویر منتشر, 3)","(حضور پیدا, 4)"
4,"(بیمارستان بستری, 3)","(نسبتا شدیدی, 2)","(غزه آواره, 3)","(پیامی توییتری, 2)","(دست پیدا, 4)"
5,"(بسیار خوب, 3)","(خودرو محقق, 2)","(غزه یکپارچه, 2)","(خیلی شدید, 2)","(اعدام محکوم, 4)"
6,"(آسیا مشخص, 3)","(سکه همزمان, 2)","(کارگزاران ممکن, 2)","(خیلی بد, 2)","(تهران بیهوش, 4)"
7,"(استقلال مشخص, 2)","(بسیار زیادی, 2)","(مجروحان وخیم, 2)","(کشور بارانی, 2)","(غزه آواره, 4)"
8,"(بی‌جهت بزرگ, 2)","(بنا متفاوت, 2)","(غزه بدتر, 2)","(تهران بزرگ, 2)","(تصاویر منتشر, 4)"
9,"(عربستان صادر, 2)","(مسکن منتشر, 2)","(غزه منتشر, 2)","(تصاویری زیبا, 2)","(اتفاقی عجیب, 4)"


## مقایسه قید‌ها و فعل‌های مرکب

In [80]:
results = dict()
results['ورزشی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in sport_results[1]]).most_common(20)
results['اقتصادی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in economic_results[1]]).most_common(20)
results['سیاسسی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in politic_results[1]]).most_common(20)
results['اجتماعی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in society_results[1]]).most_common(20)
results['کل دسته‌بندی‌ها'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in sport_results[1] + economic_results[1] + politic_results[1] + society_results[1]]).most_common(20)
pd.DataFrame(results)

,ورزشی,اقتصادی,سیاسسی,اجتماعی,کل دسته‌بندی‌ها
0,"(کنار گذاشت#گذار, 3)","(چقدر #است, 10)","(چه بود#باش, 3)","(چه بود#باش, 4)","(چقدر #است, 12)"
1,"(عقب بود#باش, 2)","(صرفا می‌توان, 2)","(چرا باید, 3)","(بیشتر شد#شو, 2)","(چه بود#باش, 8)"
2,"(به‌دست آورد#آور, 2)","(باز گشت#گرد, 2)","(بس کرد#کن, 3)","(بیشتر #است, 2)","(چه شد#شو, 5)"
3,"(چقدر #است, 1)","(چگونه بود#باش, 2)","(پیش رفت#رو, 2)","(البته نگفته, 1)","(قبل بود#باش, 4)"
4,"(جلو کرد#کن, 1)","(چه شد#شو, 2)","(چه کرد#کن, 2)","(صرفا باید, 1)","(پیش رفت#رو, 4)"
5,"(کم کرد#کن, 1)","(قبل بود#باش, 2)","(بیرون بیاید, 2)","(بیرون آورد#آور, 1)","(چرا باید, 4)"
6,"(بیرون داد, 1)","(جلو برد#بر, 2)","(چه شد#شو, 2)","(درست بود#باش, 1)","(کنار گذاشت#گذار, 3)"
7,"(سخت #است, 1)","(پیش بود#باش, 2)","(باز شد#شو, 2)","(مدام گفت#گو, 1)","(باز شد#شو, 3)"
8,"(امسال #است, 1)","(پیش رفت#رو, 2)","(باز #است, 2)","(این‌گونه نیست, 1)","(باز کرد#کن, 3)"
9,"(باز شد#شو, 1)","(همچنان پابرجاست, 1)","(بالا رفت#رو, 1)","(عقب برمی‌گشت, 1)","(چه کرد#کن, 3)"


## مقایسه ترکیب‌های اضافی

In [86]:
results = dict()
results['ورزشی'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in sport_results[2]]).most_common(20)
results['اقتصادی'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in economic_results[2]]).most_common(20)
results['سیاسسی'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in politic_results[2]]).most_common(20)
results['اجتماعی'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in society_results[2]]).most_common(20)
results['کل دسته‌بندی‌ها'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in sport_results[2] + economic_results[2] + politic_results[2] + society_results[2]]).most_common(20)
pd.DataFrame(results)

,ورزشی,اقتصادی,سیاسسی,اجتماعی,کل دسته‌بندی‌ها
0,"(کریستیانو رونالدو, 17)","(درصد افزایش, 4)","(هدف قرار, 16)","(آتیلا پسیانی, 13)","(کریستیانو رونالدو, 17)"
1,"(یحیی گل‌محمدی, 15)","(خودرو حذف, 4)","(حسین امیرعبداللهیان, 6)","(کشور خبر, 7)","(هدف قرار, 17)"
2,"(سردار آزمون, 10)","(فرزند افزایش, 3)","(علی مطهری, 6)","(زهرا نژادبهرام, 6)","(واکنش نشان, 16)"
3,"(واکنش نشان, 9)","(بررسی‌ها نشان, 3)","(بنیامین نتانیاهو, 6)","(امام خمینی, 4)","(یحیی گل‌محمدی, 15)"
4,"(علیرضا بیرانوند, 9)","(مرغ اعلام, 3)","(آیت‌الله خامنه‌ای, 5)","(درجه‌ای دما, 4)","(آتیلا پسیانی, 13)"
5,"(کریس رونالدو, 6)","(درصدی همراه, 3)","(محمدرضا باهنر, 5)","(رضا کیانیان, 3)","(سردار آزمون, 10)"
6,"(علیرضا دبیر, 6)","(رشیدی کوچی, 3)","(غزه خبر, 5)","(واکنش نشان, 3)","(علیرضا بیرانوند, 9)"
7,"(دیوید تیلور, 5)","(حالی رقم, 3)","(علیرضا پناهیان, 5)","(سحر قریشی, 3)","(امام خمینی, 8)"
8,"(شهربانو منصوریان, 5)","(لیتر بنزین, 3)","(علم الهدی, 4)","(آبان ماه, 3)","(رشیدی کوچی, 7)"
9,"(امیر قلعه‌نویی, 5)","(کشور وارد, 2)","(آنتونی بلینکن, 4)","(توجه قرار, 3)","(آیت‌الله خامنه‌ای, 7)"


## نتیجه گیری 

<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
</head>
<body>
    <p dir="rtl">
        با توجه به توزیع ترکیب‌های اضافی، اغلب در اخبار ورزشی اسم بازیکنان و موراد مشابه زیاد تکرار شده به همین دلیل به صورت کلی در این مقایسه بخش ورزشی شامل عبارات پر تکرار تر هست. 
        اما ترکیب‌های دیگر، دسته‌ی غالبی ندارند.
        همینطور با توجه به اینکه در نتایج نهایی، بخش اول افعال مرکب،  به صورت فعل حساب نشده‌اند. به همین دلیل در ترکیب‌های وصفی و .. مقداری مشکلات وجود دارد.
    </p>
</body>
</html>

## انجام آزمایش با حذف حروف اضافه از کلمات بعد از توکنایز کلمات

In [93]:
def extract_phrases_with_deleting_stop_words(data):
    news_normalized = [normalizer.normalize(item)for item in data]
    news_sentences = [sent_tokenize(item) for item in news_normalized]
    news_tokens = [[[ t for t in word_tokenize(sent) if t not in stopwords] for sent in sents] for sents in news_sentences]
    mosufs = []
    gheids = []
    ezafi = []
    for sentences in news_tokens:
      for sentence in sentences:
        tree=vasf_ext(tagger.tag([word for word in sentence]))
        for subtree in tree.subtrees():
            if subtree.label() == 'NVASF':
                mosufs.append(subtree.leaves())
            if subtree.label() == 'EZAFI':
                ezafi.append(subtree.leaves())  
    
    for sentences in news_tokens:
      for sentence in sentences:
        tree=vasf_ext(tagger.tag([lemmatizer.lemmatize(word) for word in sentence]))
        for subtree in tree.subtrees():
            if subtree.label() == 'GHEIDV':
                gheids.append(subtree.leaves())
    return mosufs, gheids, ezafi

In [94]:
sport_results = extract_phrases_with_deleting_stop_words(sport_data)
economic_results = extract_phrases_with_deleting_stop_words(economic_data)
politic_results = extract_phrases_with_deleting_stop_words(politic_data)
society_results = extract_phrases_with_deleting_stop_words(society_data)

In [95]:
results = dict()
results['ورزشی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in sport_results[0]]).most_common(10)
results['اقتصادی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in economic_results[0]]).most_common(10)
results['سیاسسی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in politic_results[0]]).most_common(10)
results['اجتماعی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in society_results[0]]).most_common(10)
results['کل دسته‌بندی‌ها'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in sport_results[0] + economic_results[1] + politic_results[0] + society_results[0]]).most_common(10)
pd.DataFrame(results)

,ورزشی,اقتصادی,سیاسسی,اجتماعی,کل دسته‌بندی‌ها
0,"(لیونل مسی, 4)","(تقریبا بی‌اثر, 2)","(غزه متوقف, 4)","(مقاله همراه, 4)","(لیونل مسی, 4)"
1,"(خیلی‌ها بی‌جهت, 2)","(مبالغی برخوردار, 2)","(غزه منتشر, 3)","(تازگی منتشر, 3)","(غزه متوقف, 4)"
2,"(عربستان صادر, 2)","(نسبتا شدیدی, 2)","(سوال مطرح, 2)","(اتفاقی عجیب, 2)","(مقاله همراه, 4)"
3,"(استقلال منتفی, 2)","(خودرو محقق, 2)","(غزه بدتر, 2)","(مهرجویی بد, 2)","(تصاویری زیبا, 3)"
4,"(پرسپولیسی‌ها تازه, 2)","(مسکن منتشر, 2)","(اسرائیل متوقف, 2)","(تصاویری زیبا, 2)","(غزه منتشر, 3)"
5,"(آمیزی منتشر, 2)","(درصد کمتر, 2)","(غزه آواره, 2)","(فریبا عادلخواه, 2)","(غزه آواره, 3)"
6,"(نبردی جذاب, 1)","(روندی کاهشی, 2)","(وعده مبنی, 2)","(همسرش منتشر, 2)","(نفر مجروح, 3)"
7,"(تراکتور مردود, 1)","(ایران آزاد, 2)","(اسرائیل +عکس, 2)","(زن عصبانی, 2)","(تازگی منتشر, 3)"
8,"(گذشته دشوارتر, 1)","(بعدا پولدار, 2)","(مدرکی مبنی, 2)","(تصاویر منتشر, 2)","(خیلی‌ها بی‌جهت, 2)"
9,"(گل‌محمدی خوش, 1)","(کار برگزار, 2)","(ادعا‌ها مبنی, 2)","(نفر مجروح, 2)","(اینستاگرامش منتشر, 2)"


##  نتیجه گیری نهایی
<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
</head>
<body>
    <p dir="rtl">
همانطور که مشاهده میکنید، با حذف حروف اضافه از بین حروف، نتایج ترکیب‌های وصفی قدری بهبود یافتند این به دلیل اشتباه افتادن برخی حروف اضافه مانند کلمه بیش، بود.
          </p>
</body>
</html>